In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt

from sklearn.utils import shuffle

import random

import tensorflow as tf
from keras.optimizers import *
from keras.layers import *
from keras.models import *
from keras.regularizers import l2

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

NUM_CLASSES = 5


# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


In [2]:
def loadfiles():
    imgs = []
    labels = []
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            file = os.path.join(dirname, filename)
            ext = filename.split('.')[1]
            if ext == 'csv':
                label = int(filename.split('_')[1])
                imgs.append(np.loadtxt(open(file, "rb"), delimiter=",", skiprows=1))
                labels.append(label)
    return np.array(imgs), np.array(labels)
            

In [3]:
imgs, labels = loadfiles()
print(imgs.shape)
indices = np.array(list(range(imgs.shape[0])))
np.random.shuffle(indices)
test_imgs = imgs[indices[:300]]
test_labels = labels[indices[:300]]
train_imgs = imgs[indices[300:]]
train_labels = labels[indices[300:]]

(1326, 127, 100)


In [4]:
print(test_imgs)

[[[1.14507884e-01 1.18617021e-01 1.85893532e-02 ... 3.89510453e-01
   5.37579775e-01 5.07201552e-01]
  [7.78835788e-02 4.01288792e-02 8.20207410e-03 ... 1.32019907e-01
   2.05461353e-01 1.62511840e-01]
  [5.67057095e-02 2.92171538e-02 5.97179122e-03 ... 9.61214453e-02
   1.49592921e-01 1.18322112e-01]
  ...
  [1.08691696e-02 2.88994727e-03 7.93149462e-04 ... 6.69958081e-06
   1.10582932e-05 3.48337790e-06]
  [5.55258989e-03 1.30137184e-03 1.33560286e-04 ... 6.01842294e-06
   2.50285711e-06 2.21470168e-06]
  [1.67713108e-04 9.05976849e-05 9.29192902e-06 ... 6.20881337e-07
   3.21128283e-07 1.75063036e-07]]

 [[3.99187833e-01 3.88671868e-02 7.86780193e-02 ... 1.04915433e-01
   6.08131997e-02 5.43594733e-02]
  [2.00505719e-01 6.40276134e-01 6.74976110e-01 ... 3.53546068e-02
   4.30643484e-02 1.54746734e-02]
  [1.45984814e-01 4.66174185e-01 4.91438657e-01 ... 2.57410873e-02
   3.13544199e-02 1.12668462e-02]
  ...
  [6.33579532e-07 2.31467723e-07 1.62229080e-05 ... 2.72951411e-06
   1.81896

In [5]:
train_arr = [[] for i in range(NUM_CLASSES)]
for idx, i in enumerate(train_labels):
    train_arr[i].append(idx)
test_arr = [[] for i in range(NUM_CLASSES)]
for idx, i in enumerate(test_labels):
    test_arr[i].append(idx)

In [6]:
def get_batch(batch_size, s="train"):
    global NUM_CLASSES, train_imgs, test_imgs, train_labels, test_labels, train_arr, test_arr
    if s == "train":
        imgs = train_imgs
        labels = train_labels
        arr = train_arr
    else:
        imgs = test_imgs
        labels = test_labels
        arr = test_arr
        
    
    n_classes = NUM_CLASSES
    n_examples, w, h = imgs.shape
    pairs = [np.zeros((batch_size, w, h, 1)) for i in range(2)]
    targets = np.zeros((batch_size,))
    
    targets[batch_size//2:] = 1
    
    categories = np.random.randint(0, n_classes, batch_size)
    
    for i in range(batch_size):
        category = categories[i]
        idx_1 = random.choice(arr[category])
        pairs[0][i,:,:,:] = imgs[idx_1].reshape(w, h, 1)
        if i >= batch_size // 2:
            category_2 = category  
        else: 
            category_2 = (category + np.random.randint(1,n_classes)) % n_classes
        idx_2 = random.choice(arr[category_2])
        
        pairs[1][i,:,:,:] = imgs[idx_2].reshape(w, h, 1)
    
    return pairs, targets
        
    
    

In [7]:
def generate(batch_size, s="train"):
    while True:
        pairs, targets = get_batch(batch_size, s)
        yield (pairs, targets)

In [8]:
def get_siamese_model(input_shape):
    """
        Model architecture
    """
    
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    # Convolutional Neural Network
    model = Sequential()
    model.add(Conv2D(64, (10,10), activation='relu', input_shape=input_shape,
                   kernel_initializer='random_normal', kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (7,7), activation='relu',
                     kernel_initializer='random_normal',
                     bias_initializer='random_normal', kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (4,4), activation='relu', kernel_initializer='random_normal',
                     bias_initializer='random_normal', kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(256, (4,4), activation='relu', kernel_initializer='random_normal',
                     bias_initializer='random_normal', kernel_regularizer=l2(2e-4)))
    model.add(Flatten())
    model.add(Dense(4096, activation='sigmoid',
                   kernel_regularizer=l2(1e-3),
                   kernel_initializer='random_normal',bias_initializer='random_normal'))
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid',bias_initializer='random_normal')(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net

In [9]:
model = get_siamese_model(tuple(list(train_imgs[0].shape)+[1]))
optimizer = Adam(lr = 0.00006)
model.compile(loss="binary_crossentropy",optimizer=optimizer)
    
model.fit(generate(32), epochs=300, steps_per_epoch=100)

Epoch 1/300
100/100 [==============================] - 11s 108ms/step - loss: 99.6061
Epoch 2/300
100/100 [==============================] - 6s 58ms/step - loss: 87.5587
Epoch 3/300
100/100 [==============================] - 6s 58ms/step - loss: 77.2512
Epoch 4/300
100/100 [==============================] - 6s 58ms/step - loss: 68.3891
Epoch 5/300
100/100 [==============================] - 6s 58ms/step - loss: 60.7178
Epoch 6/300
100/100 [==============================] - 6s 58ms/step - loss: 54.0565
Epoch 7/300
100/100 [==============================] - 6s 58ms/step - loss: 48.2529
Epoch 8/300
100/100 [==============================] - 6s 57ms/step - loss: 43.1871
Epoch 9/300
100/100 [==============================] - 6s 58ms/step - loss: 38.7755
Epoch 10/300
100/100 [==============================] - 6s 58ms/step - loss: 34.8996
Epoch 11/300
100/100 [==============================] - 6s 59ms/step - loss: 31.5141
Epoch 12/300
100/100 [==============================] - 6s 58ms/step - l

In [10]:
model.save_weights('epoch300-weights-new.h5')

In [11]:
def make_oneshot_task(N, s="val"):
    global NUM_CLASSES, train_imgs, test_imgs, train_labels, test_labels, train_arr, test_arr
    if s == "train":
        imgs = train_imgs
        labels = train_labels
        arr = train_arr
    else:
        imgs = test_imgs
        labels = test_labels
        arr = test_arr
        
    n_classes = NUM_CLASSES
    n_examples, w, h = imgs.shape
    
    classes = list(range(NUM_CLASSES))

    random.shuffle(classes)
    
    true_class = classes[0]
    
    idx1, idx2 = np.random.choice(arr[true_class], size=(2,))
    
    test_image = np.asarray([imgs[idx1,:,:]]*N).reshape(N, w, h, 1)
    
    ssidxs = [idx2]
    for i in range(1, NUM_CLASSES):
        ssidxs.append(np.random.choice(arr[classes[i]]))
    
    support_set = imgs[ssidxs,:,:]
    
    support_set = support_set.reshape(N, w, h, 1)
    
    targets = np.zeros((N,))
    targets[0] = 1
    
    test_image, support_set, targets = shuffle(test_image, support_set, targets)
    
    
    pairs = [test_image, support_set]
    
    return pairs, targets, classes


In [12]:
def test_oneshot(model, N, k, s = "val", verbose = 0):
    global NUM_CLASSES
    n_correct = 0
    total = [0 for i in range(NUM_CLASSES)]
    correct = [0 for i in range(NUM_CLASSES)]
    if verbose:
        print("Evaluating model on {} random {} way one-shot learning tasks ... \n".format(k,N))
    for i in range(k):
        inputs, targets, classes = make_oneshot_task(N,s)
        total[classes[np.argwhere(targets == 1)[0,0]]]+=1
        probs = model.predict(inputs)
        if np.argmax(probs) == np.argmax(targets):
            n_correct+=1
            correct[classes[np.argwhere(targets == 1)[0,0]]]+=1
    percent_correct = (100.0 * n_correct / k)
    if verbose:
        print("Got an average of {}% {} way one-shot learning accuracy \n".format(percent_correct,N))
    percentages = [(100.0*a)/b for a, b in zip(correct, total)]
    return percentages, percent_correct

In [13]:
test_oneshot(model, 5, 1000, verbose=1)

Evaluating model on 1000 random 5 way one-shot learning tasks ... 

Got an average of 67.9% 5 way one-shot learning accuracy 



([64.79591836734694,
  66.5158371040724,
  70.0,
  69.79166666666667,
  68.50828729281768],
 67.9)